In [3]:
import sys
import time
import networkx as nx
import pandas as pd
code_path = '../ura'
sys.path.append(code_path)
import create_graph
import stat_analysis
reload(create_graph)
reload(stat_analysis)

<module 'stat_analysis' from '../ura\stat_analysis.pyc'>

In [4]:
TF_list = create_graph.create_TF_list(slowkow_bool=True,
                    slowkow_files=['../slowkow_databases/TRED_TF.txt',
                                   '../slowkow_databases/ITFP_TF.txt',
                                   '../slowkow_databases/ENCODE_TF.txt',
                                   '../slowkow_databases/Neph2012_TF.txt',
                                   '../slowkow_databases/TRRUST_TF.txt',
                                   '../slowkow_databases/Marbach2016_TF.txt'],
                    slowkow_sep = '\n',
                    jaspar_bool=True,
                    jaspar_file="../jaspar_genereg_matrix.txt")
TF_list.extend(['TNF','LBP','IFNG', 'IL1B'])
len(TF_list)

3987

In [5]:
unfiltered_DG = create_graph.load_STRING_to_digraph(filename = "../9606.protein.actions.v10.5.txt", confidence_filter=400)
DG = create_graph.filter_digraph(unfiltered_DG, TF_list)

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-7374...done.
Finished.
312 input query terms found no hit:
	[u'ENSP00000376684', u'ENSP00000289352', u'ENSP00000202788', u'ENSP00000373637', u'ENSP00000367802',
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


## HUVEC ##

In [6]:
filename_huvec = '../geo2r_GSE2639_huvec.txt'
huvec_genes = ['TNF','IFNG','LBP', 'NFKB1', 'NFKB2', 'REL', 'RELA', 'RELB', 'PCBP3', 'PCBP2', 'PCBP1', 'PCBP4', 'NFKBIA']
DEG_list_huvec, DEG_to_pvalue_huvec, DEG_to_updown_huvec = create_graph.create_DEG_list(filename_huvec, p_value_filter = 0.3)
DG_huvec = create_graph.add_updown_from_DEG(DG, DEG_to_updown_huvec)

In [7]:
not_biased_zscores_huvec = stat_analysis.tr_zscore(DG_huvec, DEG_list_huvec, auto_correct_ = False, correct_for_bias = False, bias_filter = 0.25)
biased_zscores_huvec = stat_analysis.tr_zscore(DG_huvec, DEG_list_huvec, auto_correct_bias = False, correct_for_bias = True, bias_filter = 0.25)
print 'bias: ' + str(stat_analysis.calculate_bias(DG_huvec))

TypeError: tr_zscore() got an unexpected keyword argument 'auto_correct_'

In [ ]:
stat_analysis.top_values(not_biased_zscores_huvec, activating = True, absolute_value = False, top = 20)

In [ ]:
print 'Total: ' + str(len(not_biased_zscores_huvec))
stat_analysis.rank(not_biased_zscores_huvec, huvec_genes, activating = True, absolute_value = False)

In [ ]:
stat_analysis.top_values(biased_zscores_huvec, activating = True, absolute_value = False, top = 20)

In [ ]:
print 'Total: ' + str(len(biased_zscores_huvec))
stat_analysis.rank(biased_zscores_huvec, huvec_genes, activating = True, absolute_value = False)

## Breast Cancer ###

In [ ]:
filename_brca = '../geo2r_GSE11352_brca_48hours.txt'
brca_genes = ['IL1B','NCOA3','PGR', 'EGR1', 'HIF1A', 'NR3C1','CTNNB1','TP53','SMARCE1','STAT5B','ESR1']
DEG_list_brca, DEG_to_pvalue_brca, DEG_to_updown_brca = create_graph.create_DEG_list(filename_brca, p_value_filter = 0.05)
DG_brca = create_graph.add_updown_from_DEG(DG, DEG_to_updown_brca)

In [ ]:
z_scores_brca = stat_analysis.tr_zscore(DG_brca, DEG_list_brca, auto_correct = True, bias_filter = 0.25)
print 'bias: ' + str(stat_analysis.calculate_bias(DG_brca))

In [ ]:
stat_analysis.top_values(z_scores_brca, activating = False, absolute_value = False, top = 1360)

In [ ]:
print 'Total: ' + str(len(z_scores_brca))
stat_analysis.rank(z_scores_brca, brca_genes, activating = False, absolute_value = False)

In [ ]:
def top_values(my_series, activating = True, absolute_value = False, top = 10):

    # top activating and inhibiting, sort by strongest zscore or log(pvalue)
    if absolute_value == True:
        top_series_abs = my_series.abs().sort_values(ascending=False).head(top)
        top_genes = list(top_series_abs.index)
        top_values = [my_series[gene] for gene in top_genes]
        return pd.Series(top_values, index=top_genes)

    # top activating
    if activating == True:
        return my_series.sort_values(ascending=False).head(top)

    # top inhibiting
    else:
        return my_series.sort_values(ascending=True).head(top)


def rank(series, genes_to_rank, activating=True, absolute_value=False, print_to_stdout=False):
    if absolute_value == True:
        sorted_dict = sorted(dict(series.abs()).items(), key=lambda x: x[1], reverse=True)
    else:
        if activating == True:
            sorted_dict = sorted(dict(series).items(), key=lambda x: x[1], reverse=True)
        else:
            sorted_dict = sorted(dict(series).items(), key=lambda x: x[1], reverse=False)

    genes = zip(*sorted_dict)[0]
    index = range(len(sorted_dict))
    gene_to_index = dict(zip(genes, index))

    return pd.Series({k: gene_to_index.get(k, None) for k in genes_to_rank}).sort_values(ascending=True)

In [ ]:
sorted_dict = sorted(dict(z_scores_brca).items(), key=lambda x: x[1], reverse=True)
sorted_dict